### LAB 5 [WORKING COPY]

In [1]:
import sklearn
import matplotlib.pyplot as plt
from typing import Tuple
from typing import List

%matplotlib inline

import numpy as np
import pandas as pd
import scipy
import pandas as pd 
from sklearn import linear_model
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, classification_report, precision_recall_curve, average_precision_score
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from sklearn.metrics import precision_recall_curve
from sklearn.ensemble import GradientBoostingClassifier

In [17]:
# df_2017 = pd.read_csv("data/2017.csv.gz",low_memory=False, encoding = "ISO-8859-1", nrows=100000, header=None)

df_2017 = pd.read_csv("data/2017.csv.gz", compression='gzip',low_memory=False, encoding = "ISO-8859-1", nrows=100000, header=None)


df_2017.head()
df_2017.reset_index()
df_2017.head(10)

,0,1,2,3,4,5,6,7
0,US1MNWR0029,20170101,PRCP,0,NaN,NaN,N,NaN
1,CA1MB000296,20170101,PRCP,0,NaN,NaN,N,NaN
2,ASN00015643,20170101,TMAX,274,NaN,NaN,a,NaN
3,ASN00015643,20170101,TMIN,218,NaN,NaN,a,NaN
4,ASN00015643,20170101,PRCP,2,NaN,NaN,a,NaN
5,US1MNHN0184,20170101,PRCP,0,NaN,NaN,N,NaN
6,US1MNHN0184,20170101,SNOW,0,NaN,NaN,N,NaN
7,US1MNHN0184,20170101,SNWD,274,NaN,NaN,N,NaN
8,US1MNCV0008,20170101,PRCP,0,NaN,NaN,N,NaN
9,US1MNCV0008,20170101,SNOW,0,NaN,NaN,N,NaN


In [18]:
df_2017.rename(index=int,columns = {0:'ID',1:'date', 2:'element',3:'data_value',4:'mFlag',5:'qFlag',6:'sFlag',7:'obsTime'}, inplace=True)

In [19]:
df_2017.head()

,ID,date,element,data_value,mFlag,qFlag,sFlag,obsTime
0,US1MNWR0029,20170101,PRCP,0,NaN,NaN,N,NaN
1,CA1MB000296,20170101,PRCP,0,NaN,NaN,N,NaN
2,ASN00015643,20170101,TMAX,274,NaN,NaN,a,NaN
3,ASN00015643,20170101,TMIN,218,NaN,NaN,a,NaN
4,ASN00015643,20170101,PRCP,2,NaN,NaN,a,NaN


In [20]:
column_1 = df_2017.ID.values
column_2 = df_2017.date.values


X = np.array(list(zip(column_1,column_2)))
X

array([['US1MNWR0029', '20170101'],
       ['CA1MB000296', '20170101'],
       ['ASN00015643', '20170101'],
       ..., 
       ['US1OHLR0003', '20170102'],
       ['US1OHLR0003', '20170102'],
       ['ASN00010036', '20170102']], 
      dtype='<U11')

In [21]:
df_2017['merged_index'] = str(df_2017['ID'])+str(df_2017['date'])

In [48]:
df_2017['data_value'].describe()

count    100000.000000
mean         57.399260
std         276.814149
min        -565.000000
25%           0.000000
50%           0.000000
75%          51.000000
max       10185.000000
Name: data_value, dtype: float64

In [23]:
df_unstacked = df_2017.pivot( columns='element', values='data_value')

In [39]:
# frames = [0, 1]
# #test = pd.concat([df_2017[[0]], df_2017[[1]]])
# #test.head()

# df['merged_index'] = pd.merge([df_2017[[0]], df_2017[[1]]], left_index=True)


In [25]:
df_unstacked.head()

element,AWDR,AWND,DAPR,DATN,DATX,DWPR,EVAP,MDPR,MDTN,MDTX,...,WSFI,WT01,WT02,WT03,WT04,WT05,WT06,WT08,WT09,WT11
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
#df_2017['merged'] = df_2017[['ID', 'date']].apply(lambda x: ''.join(str(x)), axis = 0)
df_2017['merged'] = df_2017['ID'].combine_first(df_2017['date'])

In [29]:
df_idd = pd.DataFrame(df_2017[['ID', 'date']])
df_idd.head()

,ID,date
0,US1MNWR0029,20170101
1,CA1MB000296,20170101
2,ASN00015643,20170101
3,ASN00015643,20170101
4,ASN00015643,20170101


In [30]:
df_2017['merged'] = df_2017[['ID','date']].apply(lambda x : '{}{}'.format(x[0],x[1]), axis=1)


In [31]:
df_2017.head()

,ID,date,element,data_value,mFlag,qFlag,sFlag,obsTime,merged_index,merged
0,US1MNWR0029,20170101,PRCP,0,NaN,NaN,N,NaN,0 US1MNWR0029\n1 CA1MB000296\n2 ...,US1MNWR002920170101
1,CA1MB000296,20170101,PRCP,0,NaN,NaN,N,NaN,0 US1MNWR0029\n1 CA1MB000296\n2 ...,CA1MB00029620170101
2,ASN00015643,20170101,TMAX,274,NaN,NaN,a,NaN,0 US1MNWR0029\n1 CA1MB000296\n2 ...,ASN0001564320170101
3,ASN00015643,20170101,TMIN,218,NaN,NaN,a,NaN,0 US1MNWR0029\n1 CA1MB000296\n2 ...,ASN0001564320170101
4,ASN00015643,20170101,PRCP,2,NaN,NaN,a,NaN,0 US1MNWR0029\n1 CA1MB000296\n2 ...,ASN0001564320170101


In [174]:
df_2017_filtered = df_2017.drop(['ID', 'date','data_value','merged_index','element'], axis=1)
df_2017_filtered.set_index('merged', inplace=True)

In [32]:
df_unstacked = df_2017.pivot( index='merged', columns='element', values='data_value')

In [33]:
df_unstacked.head()

element,AWDR,AWND,DAPR,DATN,DATX,DWPR,EVAP,MDPR,MDTN,MDTX,...,WSFI,WT01,WT02,WT03,WT04,WT05,WT06,WT08,WT09,WT11
merged,,,,,,,,,,,,,,,,,,,,,
AE00004119620170102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AFM0004094820170101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AFM0004094820170102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AFM0004099020170102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AG00006039020170102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [169]:
print(df_unstacked.shape)
print(df_2017.shape)

(32475, 62)
(100000, 10)


In [34]:
df_unstacked_core = df_unstacked[[ 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN']]

In [35]:
df_unstacked_core.head()

element,PRCP,SNOW,SNWD,TMAX,TMIN
merged,,,,,
AE00004119620170102,NaN,NaN,NaN,264.0,NaN
AFM0004094820170101,NaN,NaN,NaN,NaN,NaN
AFM0004094820170102,NaN,NaN,NaN,NaN,NaN
AFM0004099020170102,NaN,NaN,NaN,NaN,NaN
AG00006039020170102,NaN,NaN,NaN,NaN,NaN


In [36]:
df_unstacked_core.isnull().sum()

element
PRCP     2655
SNOW    20071
SNWD    21607
TMAX    20035
TMIN    20050
dtype: int64

In [37]:
len(df_unstacked_core)

32475

#### Mahmoud 

In [74]:
df_unstacked['TMIN'].isnull().sum()

20050

In [75]:
df_unstacked['TMAX'].isnull().sum()

20035

In [40]:
df_unstacked.shape

(32475, 62)

In [69]:
df_2017.shape

(100000, 10)

In [94]:
df_unstacked['TMIN'].describe()

count    12425.000000
mean       -38.104467
std        108.681028
min       -565.000000
25%       -101.000000
50%        -50.000000
75%         19.000000
max        297.000000
Name: TMIN, dtype: float64

In [83]:
temp_df = df_unstacked.query(('TMAX >= 0 or TMAX <0') or ('TMIN >=0 or TMIN <0'))

In [90]:
temp_df['TMIN'] = temp_df['TMIN'].fillna(temp_df['TMIN'].mean())

C:\Users\hamsh\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [239]:
temp_df = pd.merge(temp_df, df_2017_filtered, right_index=True, left_index=True)

In [241]:
temp_df.columns

Index(['AWDR', 'AWND', 'DAPR', 'DATN', 'DATX', 'DWPR', 'EVAP', 'MDPR', 'MDTN',
       'MDTX', 'MNPN', 'MXPN', 'PGTM', 'PRCP', 'PSUN', 'SN31', 'SN32', 'SN33',
       'SN35', 'SN36', 'SN51', 'SN52', 'SN53', 'SN55', 'SN56', 'SNOW', 'SNWD',
       'SX31', 'SX32', 'SX33', 'SX35', 'SX36', 'SX51', 'SX52', 'SX53', 'SX55',
       'SX56', 'TAVG', 'THIC', 'TMAX', 'TMIN', 'TOBS', 'TSUN', 'WDF2', 'WDF5',
       'WDFG', 'WDMV', 'WESD', 'WESF', 'WSF2', 'WSF5', 'WSFG', 'WSFI', 'WT01',
       'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT08', 'WT09', 'WT11', 'mFlag',
       'qFlag', 'sFlag', 'obsTime'],
      dtype='object')

In [252]:
temp_df.columns
columns = ['AWDR', 'AWND', 'DAPR', 'DATN', 'DWPR', 'EVAP', 'MDPR', 'MDTN', 'MNPN', 'MXPN', 'PGTM', 'PRCP', 'PSUN', 'SN31', 'SN32', 'SN33',
       'SN35', 'SN36', 'SN51', 'SN52', 'SN53', 'SN55', 'SN56', 'SNOW', 'SNWD',
       'SX31', 'SX32', 'SX33', 'SX35', 'SX36', 'SX51', 'SX52', 'SX53', 'SX55',
       'SX56', 'TAVG', 'THIC', 'TMAX', 'TMIN', 'TOBS', 'TSUN', 'WDF2', 'WDF5',
       'WDFG', 'WDMV', 'WESD', 'WESF', 'WSF2', 'WSF5', 'WSFG', 'WSFI', 'WT01',
       'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT08', 'WT09', 'WT11', 'obsTime']

In [253]:
y = temp_df['TMIN']
X = temp_df[columns]

In [254]:
X.columns

Index(['AWDR', 'AWND', 'DAPR', 'DATN', 'DWPR', 'EVAP', 'MDPR', 'MDTN', 'MNPN',
       'MXPN', 'PGTM', 'PRCP', 'PSUN', 'SN31', 'SN32', 'SN33', 'SN35', 'SN36',
       'SN51', 'SN52', 'SN53', 'SN55', 'SN56', 'SNOW', 'SNWD', 'SX31', 'SX32',
       'SX33', 'SX35', 'SX36', 'SX51', 'SX52', 'SX53', 'SX55', 'SX56', 'TAVG',
       'THIC', 'TMAX', 'TMIN', 'TOBS', 'TSUN', 'WDF2', 'WDF5', 'WDFG', 'WDMV',
       'WESD', 'WESF', 'WSF2', 'WSF5', 'WSFG', 'WSFI', 'WT01', 'WT02', 'WT03',
       'WT04', 'WT05', 'WT06', 'WT08', 'WT09', 'WT11', 'obsTime'],
      dtype='object')

In [255]:
X.fillna(X.mean(), inplace=True)

C:\Users\hamsh\Anaconda3\lib\site-packages\pandas\core\generic.py:3549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,AWDR,AWND,DAPR,DATN,DWPR,EVAP,MDPR,MDTN,MNPN,MXPN,...,WT01,WT02,WT03,WT04,WT05,WT06,WT08,WT09,WT11,obsTime
merged,,,,,,,,,,,,,,,,,,,,,
AE00004119620170102,83.380952,36.758737,2.709402,2.0,2.0,20.732441,86.842105,186.228571,63.111111,141.046595,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1110.70957
AE00004119620170102,83.380952,36.758737,2.709402,2.0,2.0,20.732441,86.842105,186.228571,63.111111,141.046595,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1110.70957
AGM0006046120170102,83.380952,36.758737,2.709402,2.0,2.0,20.732441,86.842105,186.228571,63.111111,141.046595,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1110.70957
AGM0006046120170102,83.380952,36.758737,2.709402,2.0,2.0,20.732441,86.842105,186.228571,63.111111,141.046595,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1110.70957
AGM0006046120170102,83.380952,36.758737,2.709402,2.0,2.0,20.732441,86.842105,186.228571,63.111111,141.046595,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1110.70957
AGM0006046120170102,83.380952,36.758737,2.709402,2.0,2.0,20.732441,86.842105,186.228571,63.111111,141.046595,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1110.70957
AM00003771120170102,83.380952,36.758737,2.709402,2.0,2.0,20.732441,86.842105,186.228571,63.111111,141.046595,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1110.70957
AM00003771120170102,83.380952,36.758737,2.709402,2.0,2.0,20.732441,86.842105,186.228571,63.111111,141.046595,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1110.70957
AM00003771120170102,83.380952,36.758737,2.709402,2.0,2.0,20.732441,86.842105,186.228571,63.111111,141.046595,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1110.70957


In [257]:
X.shape

(66367, 61)

In [258]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

# feature extraction
svc = LinearRegression()
rfe = RFE(estimator=svc, n_features_to_select=10, step=100)
RFE = rfe.fit(X, y)

print(RFE)
print(RFE.n_features_)
print(RFE.support_)
print(RFE.ranking_)

RFE(estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
  n_features_to_select=10, step=100, verbose=0)
10
[False False  True False False False False False False False False False
  True  True False False  True False  True False False False False False
 False  True False False False False  True False  True False False False
 False False  True False False False False False False False  True False
 False False False False False False False False False False False False
 False]
[2 2 1 2 2 2 2 2 2 2 2 2 1 1 2 2 1 2 1 2 2 2 2 2 2 1 2 2 2 2 1 2 1 2 2 2 2
 2 1 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [259]:
selected_feature = []
rfe_list = RFE.support_
for index in range(len(rfe_list)):
    if rfe_list[index] == True:
        selected_feature.append(columns[index])
    else:
        continue

In [287]:
X.drop('TMIN',axis=1,inplace=True)

C:\Users\hamsh\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [288]:
X.describe()

,AWDR,AWND,DAPR,DATN,DWPR,EVAP,MDPR,MDTN,MNPN,MXPN,...,WT01,WT02,WT03,WT04,WT05,WT06,WT08,WT09,WT11,obsTime
count,66367.000000,66367.000000,66367.000000,66367.0,66367.0,66367.000000,66367.000000,66367.000000,66367.000000,66367.000000,...,66367.0,66367.0,66367.0,66367.0,66367.0,66367.0,66367.0,66367.0,66367.0,66367.000000
mean,83.380952,36.758737,2.709402,2.0,2.0,20.732441,86.842105,186.228571,63.111111,141.046595,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1110.709570
std,7.931292,19.788581,0.091210,0.0,0.0,3.835156,4.489144,1.070350,4.529118,5.913811,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,376.771864
min,1.000000,0.000000,2.000000,2.0,2.0,0.000000,0.000000,131.000000,-178.000000,-178.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,400.000000
25%,83.380952,36.758737,2.709402,2.0,2.0,20.732441,86.842105,186.228571,63.111111,141.046595,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,830.000000
50%,83.380952,36.758737,2.709402,2.0,2.0,20.732441,86.842105,186.228571,63.111111,141.046595,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1110.709570
75%,83.380952,36.758737,2.709402,2.0,2.0,20.732441,86.842105,186.228571,63.111111,141.046595,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1110.709570
max,360.000000,488.000000,14.000000,2.0,2.0,251.000000,429.000000,246.000000,189.000000,306.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2400.000000


In [289]:
y.shape

(66367,)

In [290]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [291]:
# Create linear regression object
regr = LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

In [292]:
print('Accuracy of Linear Regression on test set: {:.5f}'.format(regr.score(X_test, y_test)))

Accuracy of Linear Regression on test set: 0.83353


In [295]:
from sklearn.metrics import mean_squared_error, r2_score


print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

Mean squared error: 1794.24


In [303]:
import xgboost as xgb

# specify parameters via map
gbm = xgb.XGBClassifier(max_depth=20, learning_rate=0.2).fit(X_train, y_train)
predictions = gbm.predict(X_test)


In [304]:
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, predictions))

Mean squared error: 314.04
